In [17]:
import pandas as pd
df=pd.read_csv('Train.csv')
df

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
...,...,...
39995,"""Western Union"" is something of a forgotten cl...",1
39996,This movie is an incredible piece of work. It ...,1
39997,My wife and I watched this movie because we pl...,0
39998,"When I first watched Flatliners, I was amazed....",1


In [18]:
import pandas as pd
import numpy as np
import re
import nltk
import pickle

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


In [19]:
#loding data sets
train_df = pd.read_csv("Train.csv")
val_df   = pd.read_csv("Valid.csv")
test_df  = pd.read_csv("Test.csv")

# Check the first few rows
train_df.head()


,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [20]:
train_df = train_df.rename(columns={'text':'review', 'label':'sentiment'})
val_df   = val_df.rename(columns={'text':'review', 'label':'sentiment'})
test_df  = test_df.rename(columns={'text':'review', 'label':'sentiment'})


In [21]:
train_df

,review,sentiment
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
...,...,...
39995,"""Western Union"" is something of a forgotten cl...",1
39996,This movie is an incredible piece of work. It ...,1
39997,My wife and I watched this movie because we pl...,0
39998,"When I first watched Flatliners, I was amazed....",1


In [22]:
#preprocessing
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()  # lowercase
    text = re.sub(r'[^a-z\s]', '', text)  # remove punctuation/numbers
    words = text.split()
    words = [w for w in words if w not in stop_words]
    return ' '.join(words)

# Apply to all datasets
train_df['clean_review'] = train_df['review'].apply(clean_text)
val_df['clean_review']   = val_df['review'].apply(clean_text)
test_df['clean_review']  = test_df['review'].apply(clean_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bhara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
vectorizer = TfidfVectorizer(max_features=5000)

# Training set
X_train = vectorizer.fit_transform(train_df['clean_review'])
y_train = train_df['sentiment']

# Validation set
X_val = vectorizer.transform(val_df['clean_review'])
y_val = val_df['sentiment']

# Test set
X_test = vectorizer.transform(test_df['clean_review'])
y_test = test_df['sentiment']


In [24]:
model = MultinomialNB()
model.fit(X_train, y_train)


,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [26]:
y_val_pred = model.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
print(classification_report(y_val, y_val_pred))


Validation Accuracy: 0.8502
              precision    recall  f1-score   support

           0       0.86      0.84      0.85      2486
           1       0.84      0.86      0.85      2514

    accuracy                           0.85      5000
   macro avg       0.85      0.85      0.85      5000
weighted avg       0.85      0.85      0.85      5000



In [9]:
y_test_pred = model.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))


Test Accuracy: 0.8618
              precision    recall  f1-score   support

           0       0.86      0.86      0.86      2495
           1       0.86      0.87      0.86      2505

    accuracy                           0.86      5000
   macro avg       0.86      0.86      0.86      5000
weighted avg       0.86      0.86      0.86      5000



In [10]:
def predict_sentiment(review):
    review = clean_text(review)
    vector = vectorizer.transform([review])
    pred = model.predict(vector)[0]
    if pred == 1:
        return "Positive"
    else:
        return "Negative"

print(predict_sentiment("The product was amazing"))
print(predict_sentiment("Worst experience ever"))


Positive
Negative


In [11]:
pickle.dump(model, open("sentiment_model.pkl", "wb"))
pickle.dump(vectorizer, open("vectorizer.pkl", "wb"))


In [12]:
pip install streamlit


Note: you may need to restart the kernel to use updated packages.
